<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Face%20Age%20RegressionTorch/AgePredictionTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

 99% 329M/331M [00:02<00:00, 152MB/s]
100% 331M/331M [00:02<00:00, 167MB/s]


In [3]:
!unzip -qq utkface-new.zip

In [4]:
import os
import numpy as np
import pandas as pd
import cv2

In [5]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [7]:
w=h=224
batch_size=32

In [8]:
images=[]
labels=[]
for image_name in os.listdir('crop_part1')[0:7000]:
  parts=image_name.split('_')
  labels.append(int(parts[0]))
  img=cv2.imread(f'crop_part1/{image_name}')
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(img)

In [9]:
images=pd.Series(images,name='images')
ages=pd.Series(labels,name='ages')
df=pd.concat([images,ages],axis=1)
df.head()

,images,ages
0,"[[[84, 73, 67], [81, 70, 64], [81, 72, 65], [8...",80
1,"[[[154, 149, 145], [152, 147, 143], [144, 139,...",7
2,"[[[254, 252, 255], [251, 249, 252], [247, 245,...",50
3,"[[[19, 18, 14], [19, 18, 14], [20, 19, 15], [2...",41
4,"[[[250, 237, 202], [255, 242, 207], [255, 243,...",22


In [10]:
under_4=[]
for i in range(len(df)):
  if df['ages'].iloc[i]<=4:
    under_4.append(df.iloc[i])

under_4=pd.DataFrame(under_4)
under_4=under_4.sample(frac=0.3)

up_4=df[df['ages']>4]

df=pd.concat([under_4,up_4])

df=df[df['ages']<90]

In [11]:
!mkdir new

In [12]:
from numpy.ma.core import append
df=pd.DataFrame(df)
new_df=pd.DataFrame(columns=['image','age'])
for i in range(len(df)):
  new_df.loc[i]={'image':f'{i}.jpg','age':df['ages'].iloc[i]}
  df['images'].iloc[i]=cv2.resize(df['images'].iloc[i],(w,h))
  # X.append(df['images'].iloc[i])
  # Y.append(df['ages'].iloc[i])
  cv2.imwrite(f'/content/new/{i}.jpg',df['images'].iloc[i])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [52]:
from torchvision.transforms.transforms import Resize

from torch.utils.data import Dataset
from PIL import Image

class PathologyPlantsDataset(Dataset):
  """
  The Class will act as the container for our dataset. It will take your dataframe, the root path, and also the transform function for transforming the dataset.
  """
  def __init__(self, data_frame, root_dir, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform
    
  def __len__(self):
        # Return the length of the dataset
        return len(self.data_frame)
    
  def __getitem__(self, idx):
        # Return the observation based on an index. Ex. dataset[0] will return the first element from the dataset, in this case the image and the label.
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, -1]
        
        if self.transform:
            image = self.transform(image)
    
        return image, label


transform = transforms.Compose([
    # transforms.PILToTensor(),
    transforms.Resize((46,46)),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    torchvision.transforms.Normalize((0, 0, 0), (1,1,1))
])


dataset= PathologyPlantsDataset(data_frame=new_df,root_dir='/content/new', transform=transform)

In [53]:
train_data_size=int(len(dataset)*0.8)
train_set, val_set = torch.utils.data.random_split(dataset, [train_data_size, len(dataset)-train_data_size])

In [79]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,512,(3,3),(1,1),(0,0))
    self.conv2=nn.Conv2d(512,256,(3,3),(1,1),(0,0))
    self.conv3=nn.Conv2d(256,128,(3,3),(1,1),(0,0))
    self.conv4=nn.Conv2d(128,64,(3,3),(1,1),(1,1))
    self.conv5=nn.Conv2d(64,32,(3,3),(1,1),(1,1))
    self.conv6=nn.Conv2d(32,16,(3,3),(1,1),(1,1))
    self.conv7=nn.Conv2d(16,64,(3,3),(1,1),(1,1))

    self.fc1=nn.Linear(64*40*40,32)
    self.fc2=nn.Linear(32,16)
    self.fc3=nn.Linear(16,8)
    self.fc4=nn.Linear(8,1)
  
  def forward(self,x):
    x=F.relu(self.conv1(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv2(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv3(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv4(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv5(x))
    # # x=F.max_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv6(x))
    # # x=F.max_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv7(x))

    x=torch.flatten(x,start_dim=1)

    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.relu(self.fc3(x))
    x=self.fc4(x)
    x=torch.softmax(x,dim=1)
    # x=F.relu(x)
    return x

In [80]:
model=Model().to(device)

In [81]:
#hyper parameters
batch_size=16
epochs=10
lr=0.001

In [82]:
train_data_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=batch_size)

In [83]:
#compile 
optimizer=torch.optim.SGD(model.parameters(),lr=lr)
loss_function=nn.MSELoss()

In [84]:
model.train()
torch.cuda.empty_cache()
for epoch in range(epochs):

  train_loss=0.0
  for images,labels in tqdm(train_data_loader):
    images,labels=images.to(device),labels.to(device)
    optimizer.zero_grad()

    preds=model(images)
    ls=loss_function(preds,labels.float())
    ls.backward()
    optimizer.step()
    train_loss+=ls
  
  total_loss=train_loss/len(train_data_loader)
  print(f"epochs: {epoch} , loss: {total_loss}")

  0%|          | 0/294 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|█████████▉| 293/294 [00:41<00:00,  7.53it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 294/294 [00:41<00:00,  7.17it/s]


epochs: 0 , loss: 1565.457275390625


100%|██████████| 294/294 [00:40<00:00,  7.31it/s]


epochs: 1 , loss: 1573.69384765625


100%|██████████| 294/294 [00:40<00:00,  7.35it/s]


...

epochs: 4 , loss: 1573.3653564453125


100%|██████████| 294/294 [00:39<00:00,  7.54it/s]


epochs: 5 , loss: 1565.3902587890625


 34%|███▍      | 100/294 [00:13<00:26,  7.43it/s]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(),"persian.pth")